In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import Government, ReadData, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data\\IO'
gams_folder = os.getcwd()+'\\gamsmodels\\G'

Read in data:

In [2]:
dsheets = {'Production_v': data_folder+'\\IO_v.xlsx', 'Production_p': data_folder+'\\IO_p.xlsx'}
GE_data = ReadData.read_data.main(dsheets,name='GE_data',components=['domstic','trade','HH','tax','invest'],balanced_data_check=False)

*Use only production sector:*

In [3]:
# from DataBase_wheels import small_updates
# GE_data = small_updates.subset_db(GE_data,GE_data.get('s_prod'))

## *Government sector essentials*

The government sector defines the tax rates that goes into other modules. In the simplest case, we assume that taxation are in constant rates (from the perspective of the firms/consumers). This entails that three tax rates should be supplied by the government sector module:
* $tauD[t,s,n]$: The tax rate levied in the use of inputs such that $PwT[t,s,n] = Peq[t,n]+tauD[t,s,n]$ where $PwT$ is the price with taxes for sectors $s$, good $n$. The revenue from this tax is thus defined as $\sum_{(s,n)\in d\_tauD[s,n]} qD[t,s,n]*tauD[t,s,n]$.
* $tauS[t,s,n]$: The tax rate levied on the output of a sector. This defines the difference between prices before taxes and equilibrium taxes (some variations in how they are specifically used in modules). The revenue from this tax is defined as $\sum_{(s,n)\in d\_tauS[s,n]} qS[t,s,n] * tauS[t,s,n]$.
* $tauLump[t,s]$ for $s\in s\_HH$: The lump sum tax that is charged on households. For other sectors (production, investment), the lump-sum tax is computed as a part of the $tauS[t,s,n]$ tax on the sectors' outputs.

Note that this restricts us to only charge constant rates for supply/demand of any type, i.e. the marginal tax rate = average tax rate. However, nothing prevents us from creating more elaborate functions for how these prices may evolve (as functions e.g. of quantity variables). 

## *1: A simple setup for tax rates*

In the following we define a simple government sector with the following instruments:
* $tauLump[t,s]$: A lump-sum tax is levied on all sectors except (1) trade and (2) inventory investment sectors.
* $tauSflat[t,s,n]$: A flat rate charged on the supply of goods.
* $tauD[t,s,n]$: A flat rate charged on the demand of goods.

Note that while $tauD$ is considered exogenous in this case, the tax rate $tauS$ is endogenous. This should be computed, e.g. on the form:
$$\begin{align}
    tauS[t,s,n] = tauSflat[t,s,n]+tauLump[t,s]*os[t,s,n],
\end{align}$$
where $os[t,s,n]$ is the output share of good $n$ in sector $s$ at time $t$. This should, however, not be applied for the household module, where the lump sum tax rate is not shifted to the price index $PbT$. 

Furthermore, we assume for simplicity that the tax rates $tauD[t,s,n]$ are used to target the total tax payments in the first year. 

In [4]:
gm_static = Government.g_static(GE_data=GE_data,work_folder=work_folder,**{'data_folder':gams_folder,'name':'g_static'})

Run simple baseline model:

In [5]:
gm_static.write_and_run(kwargs_init={'check_variables':True})

*Adjust lump sum tax to balance IO:*

In [6]:
# gm_static.balanceIO_lumpsum(inplace=True)

*Calibrate:*

In [7]:
gm_static.setstate('SC')
gm_static.reset_settings() # this resets the RunFile and CollectFile settings.
gm_static.write_and_run(overwrite=True) # this overwrites files in the work_folder if they already exists.

*Inspect tax rates:*

In [8]:
db_static = gm_static.model_instances['baseline'].out_db
db_static.get('tauD').unstack(level='s')

s,F,HH,I_B,I_M,a,b,itory
n,,,,,,,
I_iB,NaN,NaN,NaN,NaN,0.048505,0.918514,NaN
I_iM,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
L,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
a,-0.003001,0.640323,0.16392,0.24384,0.000000,0.000000,0.025764
a_F,NaN,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
b,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
b_F,NaN,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000


### *2: Run dynamic model*

In [9]:
tindex = range(1,4)

In [10]:
GE_data = ReadData.read_data.main(dsheets,name='GE_data',components=['domstic','trade','HH','tax','invest'],balanced_data_check=False)

In [11]:
# from DataBase_wheels import small_updates
# GE_data = small_updates.subset_db(GE_data,GE_data.get('s_prod'))

In [12]:
gm = Government.g_dynamic(GE_data=GE_data,work_folder=work_folder,gs_vals = {'t': tindex},**{'data_folder':gams_folder,'name':'g_dynamic'})
gm.ivfs(db_static,merge=False) # initialize levels from static model
gm.initialize_variables(**{'check_variables':True})
gm.write_and_run(overwrite=True)

In [13]:
db = gm.model_instances['baseline'].out_db 
[db.series.__delitem__(sym) for sym in db.symbols if sym not in gm.model.database.symbols];
gm.model.database.merge_dbs(gm.model.database,db,'second')
gm.export(name='G',repo = os.getcwd()+'\\gamsmodels\\GE')

'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v4\\examples\\gamsmodels\\GE\\G'